<a href="https://colab.research.google.com/github/cristobalperezp/MDS7202-Laboratorio/blob/main/Lab_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Laboratorio 9:  ¿Superhéroe o Villano?  🦴</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Cristobal Pérez
- Nombre de alumno 2: Patricio Pérez


### **Link de repositorio de GitHub:** `https://github.com/cristobalperezp/MDS7202-Laboratorio/tree/main`

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [1]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
#!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
#!pip install umap-learn

# Librería para NLP
#!pip install nltk
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\patri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\patri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [2]:
# Si usted está utilizando Colaboratory le puede ser útil este código para cargar los archivos.
#try:
#    from google.colab import drive
#    drive.mount("/content/drive")
#    path = 'Dirección donde tiene los archivos en el Drive'
#except: 
#    print('Ignorando conexión drive-colab')

In [3]:
df_comics = pd.read_csv('https://raw.githubusercontent.com/cristobalperezp/MDS7202-Laboratorio/main/df_comics.csv')
df_comics_no_label = pd.read_csv('https://raw.githubusercontent.com/cristobalperezp/MDS7202-Laboratorio/main/comics_no_label.csv')
df_comics = df_comics.dropna(subset=['history_text']) # eliminar ejemplos sin historia

In [4]:
# queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [5]:
docs = ['The teacher rocks like a good rock & roll',
             'the rock is the best actor in the world']


docs_tokenizados = [word_tokenize(doc)  for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [6]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    def __init__(self):
        self.ps = PorterStemmer()
    def __call__(self, doc):
        doc_tok = word_tokenize(doc)
        doc_tok = [t for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador = StemmerTokenizer()

# Creamos algunos documentos
docs = ['The teacher rocks like a good rock & roll',
        'the rock is the best actor in the world',
        'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\patri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [7]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['New', 'York', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [8]:
bow = CountVectorizer(tokenizer= StemmerTokenizer())
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [9]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(), ngram_range=(1,2))
df = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,...,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,...,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [x] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [x] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [10]:
#### Código aquí ####

In [11]:
bow = CountVectorizer(tokenizer= StemmerTokenizer(),
                      #ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )

In [12]:
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

In [13]:
ct = ColumnTransformer(
    [
        ('BoW', bow, 'history_text'),
        ('Scaler', MinMaxScaler(), atributos_de_interes)
        
    ],
    remainder='passthrough',
    n_jobs=-1,
)

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [x] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [x] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [x] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [14]:
#### Código aquí ####

In [15]:
from sklearn.dummy import DummyClassifier

In [16]:
# Clasificadores
clf_nb = Pipeline(
    steps=[
        ("preprocessing", ct),
        ("selection", SelectPercentile(f_classif, percentile=90)),
        ("model", MultinomialNB()),
    ]
)

clf_base = Pipeline(
    steps=[
        ("preprocessing", ct),
        ("selection", SelectPercentile(f_classif, percentile=90)),
        ("model", DummyClassifier(strategy='stratified')),
    ]
)

In [17]:
features = ['history_text','intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']

In [18]:
# Primero se separan los datos en train y test
features = df_comics.loc[:,features]
y = df_comics.loc[:, 'alignment']

X_train, X_test, y_train, y_test = train_test_split(
    features, y, test_size=0.2, shuffle=True, stratify=y, random_state=42
)

In [19]:
#Fit
clf_nb.fit(X_train,y_train)

# Predict
y_pred = clf_nb.predict(X_train)

# Resultados
print(classification_report(y_pred, y_train))

              precision    recall  f1-score   support

         Bad       0.94      0.81      0.87       398
        Good       0.88      0.94      0.91       556
     Neutral       0.69      0.84      0.76        74

    accuracy                           0.88      1028
   macro avg       0.83      0.86      0.84      1028
weighted avg       0.89      0.88      0.88      1028



In [20]:
# Predict
y_pred = clf_nb.predict(X_test)

# Resultados
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

         Bad       0.55      0.57      0.56        83
        Good       0.80      0.70      0.75       170
     Neutral       0.00      0.00      0.00         4

    accuracy                           0.65       257
   macro avg       0.45      0.42      0.43       257
weighted avg       0.71      0.65      0.67       257



In [21]:
#Fit
clf_base.fit(X_train,y_train)

# Predict
y_pred = clf_base.predict(X_train)

# Resultados
print(classification_report(y_pred, y_train))

              precision    recall  f1-score   support

         Bad       0.32      0.34      0.33       327
        Good       0.61      0.59      0.60       607
     Neutral       0.11      0.11      0.11        94

    accuracy                           0.47      1028
   macro avg       0.35      0.35      0.35      1028
weighted avg       0.47      0.47      0.47      1028



In [22]:
# Predict
y_pred = clf_base.predict(X_test)

# Resultados
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

         Bad       0.41      0.40      0.40        88
        Good       0.59      0.59      0.59       148
     Neutral       0.13      0.14      0.14        21

    accuracy                           0.49       257
   macro avg       0.38      0.38      0.38       257
weighted avg       0.49      0.49      0.49       257



```
a. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.
Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: 
b. ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?




```

Recordemos el significado de los componentes del classification_report:

(Classification_report para set de testeo de MultinomialNB)

|           | Precision | Recall | F1-Score | Support |
|-----------|-----------|--------|----------|---------|
| Bad       | 0.55      | 0.57   | 0.56     | 83      |
| Good      | 0.80      | 0.70   | 0.75     | 170     |
| Neutral   | 0.00      | 0.00   | 0.00     | 4       |
|           |           |        |          |         |
| Accuracy  |           |        | 0.65     | 257     |
| Macro Avg | 0.45      | 0.42   | 0.43     | 257     |
| Wtd Avg   | 0.71      | 0.65   | 0.67     | 257     |


Precisión: Mide la proporción de predicciones correctas entre todas las predicciones realizadas para una determinada clase. Una precisión alta indica que el modelo ha cometido muy pocos errores de "falsos positivos" para esa clase.

Recall (o sensibilidad): Mide la proporción de instancias de una clase que el modelo ha logrado identificar correctamente. Un recall alto indica que el modelo ha cometido muy pocos errores de "falsos negativos" para esa clase.

F1-Score: Es la media armónica entre la precisión y el recall. Proporciona una única métrica que equilibra las consideraciones de precisión y recall. Un valor F1 más alto indica un mejor equilibrio entre la precisión y el recall.

Support: Es el número de instancias reales de la clase en el conjunto de prueba.

Las últimas tres filas muestran el promedio de las métricas para todas las clases. La fila "accuracy" muestra la precisión global del modelo, que es la proporción de todas las predicciones que fueron correctas. Las filas "macro avg" y "weighted avg" muestran promedios de las métricas para todas las clases, con "macro avg" calculando un promedio simple y "weighted avg" calculando un promedio ponderado por el soporte (número de instancias) de cada clase.

__Respuesta (a):__ Primero demos un contexto del modelo, Multinomial Naive Bayes (MultinomialNB) es un clasificador probabilístico basado en el teorema de Bayes con la suposición de "ingenuidad" (naive), es decir, supone que todas las características son condicionalmente independientes dada la variable de salida o la clase. Esta suposición simplifica significativamente el cálculo, aunque es claramente una suposición muy fuerte y no siempre es realista en situaciones del mundo real.

Analizando el classification_report para evaluar como se comporta el modelo en el set de testeo, el modelo tiene una precisión del 65% en la predicción correcta de las clases. Sin embargo, vale la pena destacar que el modelo no ha podido clasificar correctamente ninguna de las instancias de la clase "Neutral" (Recall y F1-Score son 0 para "Neutral").

El modelo tiene una precisión y un recall relativamente equilibrados para la clase "Bad" (0.55 y 0.57 respectivamente). Esto indica que el modelo tiene un rendimiento moderado para esta clase, comete errores tanto en falsos positivos como en falsos negativos.

El modelo tiene una alta precisión para la clase "Good" (0.80) pero un recall menor (0.70). Esto sugiere que el modelo es muy bueno evitando falsos positivos para esta clase, pero tiende a perder algunas instancias positivas (falsos negativos).

El modelo parece tener un rendimiento malo en general, con un buen desempeño para la clase "Good", un rendimiento moderado para la clase "Bad" y problemas significativos con la clase "Neutral". Este modelo podría mejorar con un enfoque de balanceo de clases, más datos, mejora  de la creación de características o un ajuste de hiperparámetros. Por ahora no recomiendo predecir la alineación de BatCow con este clasificador.

__Respuesta (b)__:

(Classification_report para set de testeo del modelo dummy estratificado)

|           | Precision | Recall | F1-Score | Support |
|-----------|-----------|--------|----------|---------|
| Bad       | 0.33      | 0.35   | 0.34     | 79      |
| Good      | 0.56      | 0.55   | 0.55     | 152     |
| Neutral   | 0.13      | 0.12   | 0.12     | 26      |
|           |           |        |          |         |
| Accuracy  |           |        | 0.44     | 257     |
| Macro Avg | 0.34      | 0.34   | 0.34     | 257     |
| Wtd Avg   | 0.44      | 0.44   | 0.44     | 257     |


El clasificador dummy estratificado es un clasificador que realiza predicciones al azar basado en la distribución de las clases en el conjunto de entrenamiento. Este tipo de clasificador se utiliza a menudo como una línea base para comparar el rendimiento de los modelos reales.

Comparando los informes de clasificación, podemos ver que el clasificador MultinomialNB tiene un mejor rendimiento que el clasificador dummy. La precisión total (accuracy) del modelo MultinomialNB es de 0.65, lo que es significativamente mayor que la precisión del modelo dummy de 0.44. Esto indica que el modelo MultinomialNB es capaz de predecir correctamente una mayor proporción de instancias en el conjunto de pruebas.

Además, el modelo MultinomialNB tiene un rendimiento superior al modelo dummy en todas las clases individuales. Por ejemplo, para la clase "Good", el modelo MultinomialNB tiene una precisión de 0.80 y un recall de 0.70, mientras que el modelo dummy tiene una precisión de 0.56 y un recall de 0.55 para la misma clase.

Por lo tanto, podemos concluir que, en general, el clasificador MultinomialNB está realizando un mejor trabajo en la predicción de las clases en comparación con el clasificador dummy. Sin embargo, el hecho de que el clasificador MultinomialNB supera al clasificador dummy no significa necesariamente que el rendimiento del MultinomialNB sea bueno. El rendimiento del modelo MultinomialNB es bastante malo, como se discutió en la pregunta anterior.

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [23]:
#### Código aquí ####

In [24]:
params = [
    # grid 1: SVM
    {
        'preprocessing__BoW__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'selection__percentile': [10, 30, 50, 70, 90],
        "model": [SVC(random_state=42)],
        'model__C': [0.1, 0.3, 0.5, 0.7, 0.9],
        'model__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'model__gamma': ['scale', 'auto'],
        'model__class_weight': [None, 'balanced']
    },
    # grid 2: Logistic Regression
    {
        'preprocessing__BoW__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'selection__percentile': [10, 30, 50, 70, 90],
        "model": [LogisticRegression(random_state=42)],
        'model__C': [0.1, 0.3, 0.5, 0.7, 0.9],
        'model__penalty': ['none', 'l2'],
        'model__solver': ['newton-cg', 'lbfgs']
    },
    # grid 3: Random Forest
    {
        'preprocessing__BoW__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'selection__percentile': [10, 30, 50, 70, 90],
        "model": [RandomForestClassifier(random_state=42)],
        "model__criterion": ["gini", "entropy"],
        'model__n_estimators': [100, 200, 300],
        'model__max_depth': [None, 5, 10, 50],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2, 4]
    }
]


In [25]:
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV

In [26]:
search = HalvingGridSearchCV(
    estimator=clf_nb,
    param_grid=params,
    verbose=10,
    scoring='f1_macro',
    cv=3,
    error_score='raise',
    n_jobs=-1,
    random_state=42
)

search.fit(X_train,y_train)

n_iterations: 4
n_required_iterations: 8
n_possible_iterations: 4
min_resources_: 18
max_resources_: 1028
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 4740
n_resources: 18
Fitting 3 folds for each of 4740 candidates, totalling 14220 fits


----------
iter: 1
n_candidates: 1580
n_resources: 54
Fitting 3 folds for each of 1580 candidates, totalling 4740 fits
----------
iter: 2
n_candidates: 527
n_resources: 162
Fitting 3 folds for each of 527 candidates, totalling 1581 fits
----------
iter: 3
n_candidates: 176
n_resources: 486
Fitting 3 folds for each of 176 candidates, totalling 528 fits


C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1173: FutureWarning: `penalty='none'`has been deprecated in 1.2 and will be removed in 1.4. To keep the past behaviour, set `penalty=None`.
  warnings.warn(
C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
C:\Users\patri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modul

HalvingGridSearchCV(cv=3, error_score='raise',
                    estimator=Pipeline(steps=[('preprocessing',
                                               ColumnTransformer(n_jobs=-1,
                                                                 remainder='passthrough',
                                                                 transformers=[('BoW',
                                                                                CountVectorizer(tokenizer=<__main__.StemmerTokenizer object at 0x000001A5B4DB3350>),
                                                                                'history_text'),
                                                                               ('Scaler',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                                 'strength_score',
                                                                                 'speed_score',
                                                                                 'durability_sc...
                                {'model': [RandomForestClassifier(random_state=42)],
                                 'model__criterion': ['gini', 'entropy'],
                                 'model__max_depth': [None, 5, 10, 50],
                                 'model__min_samples_leaf': [1, 2, 4],
                                 'model__min_samples_split': [2, 5, 10],
                                 'model__n_estimators': [100, 200, 300],
                                 'preprocessing__BoW__ngram_range': [(1, 1),
                                                                     (1, 2),
                                                                     (1, 3)],
                                 'selection__percentile': [10, 30, 50, 70,
                                                           90]}],
                    random_state=42, scoring='f1_macro', verbose=10)

In [40]:
#Reportar mejor métrica y mejores parámetros
print('f1-score:',search.best_score_)
print()

search.best_estimator_

f1-score: 0.4456824014557075



Pipeline(steps=[('preprocessing',
                 ColumnTransformer(n_jobs=-1, remainder='passthrough',
                                   transformers=[('BoW',
                                                  CountVectorizer(tokenizer=<__main__.StemmerTokenizer object at 0x000001A5BA0815D0>),
                                                  'history_text'),
                                                 ('Scaler', MinMaxScaler(),
                                                  ['intelligence_score',
                                                   'strength_score',
                                                   'speed_score',
                                                   'durability_score',
                                                   'power_score',
                                                   'combat_score'])])),
                ('selection', SelectPercentile(percentile=90)),
                ('model',
                 LogisticRegression(C=0.5, penalty='none', random_state=42))])

In [28]:
# Predict
y_pred = search.predict(X_test)

# Resultados
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

         Bad       0.51      0.63      0.56        70
        Good       0.85      0.72      0.78       176
     Neutral       0.13      0.27      0.18        11

    accuracy                           0.67       257
   macro avg       0.50      0.54      0.51       257
weighted avg       0.73      0.67      0.69       257



In [39]:
best_model = search.best_estimator_
count_vectorizer = best_model.named_steps['preprocessing'].named_transformers_['BoW']
params = count_vectorizer.get_params()
params

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': <__main__.StemmerTokenizer at 0x1a5b68d0050>,
 'vocabulary': None}

```
Escriba aquí su justificación
```

Se ha reportado la mejor combinación de hiperparámetros para el clasificador, basándose en la métrica F1-score macro. Esta métrica combina tanto la precisión como el recall en un solo número, y el "macro" indica que se ha promediado la métrica entre todas las clases sin tener en cuenta el desequilibrio de clases. Esta es una buena métrica para usar en este caso, ya que el conjunto de datos está desequilibrado (hay más instancias de la clase "Good" que de las otras clases), y podemos ver como se comporta el modelo en general.

Según la salida de search.best_estimator_, la mejor combinación de hiperparámetros encontrada es:

Preprocesamiento: El CountVectorizer utilizó StemmerTokenizer personalizado para el tokenizador. Esto sugiere que el preprocesamiento de las palabras (mediante la eliminación de las terminaciones de las palabras para reducir el número de palabras únicas y agrupar las palabras similares) tuvo un impacto positivo en el rendimiento del modelo. Se utilizó 'ngram_range': (1, 1).

Selección de características: Se seleccionó el percentil 70 en SelectPercentile. Esto significa que se conservaron las características que se encuentran en el top 70% con las puntuaciones más altas de acuerdo a la prueba que usa el método(ANOVA f-test). Esto sugiere que la selección de características tuvo un impacto positivo en el rendimiento del modelo.

Modelo: Se seleccionó LogisticRegression con un parámetro de regularización C de 0.9. Esto significa que se aplicó una regularización muy ligera al modelo, lo que sugiere que los datos son de alta calidad y no hay mucho ruido. Esto también sugiere que el modelo de regresión logística es un buen modelo para este problema, que puede ser a causa de la naturaleza lineal de los datos.

En resumen, el modelo logistico con regularización C=0.9, seleccionando el 70% de las características y usando un tokenizador personalizado con 'ngram (1,1)', obtuvo un F1-score macro de 0.67, que es un poco mejor que el F1-score macro de 0.65 obtenido en la sección 1.2. Esto sugiere que la búsqueda de hiperparámetros ha mejorado el rendimiento del modelo. Las causas de esta mejora pueden ser el preprocesamiento de las palabras, la selección de características y la regularización ligera que ayudaron a un mejor rendimiento del modelo.


## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [29]:
#### Código aquí ####

In [30]:
df_comics_no_label = df_comics_no_label.drop_duplicates(subset = ['name'])
df_comics_no_label = df_comics_no_label.dropna(subset=['history_text'])

In [31]:
Test = df_comics_no_label.loc[:, ['history_text','intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']]

In [32]:
y_pred = search.predict(Test)

df_comics_no_label['alignment'] = y_pred

In [33]:
lista = ['Vergil', 'Gorilla Girl' , 'Batcow']
mask = df_comics_no_label.name.isin(lista)
df_comics_no_label.loc[mask,['name','alignment']]

,name,alignment
16,Batcow,Good
40,Gorilla Girl,Good
78,Vergil,Neutral


<https://dc.fandom.com/wiki/Bat-Cow_(Prime_Earth)>

<https://legomarveldc.fandom.com/wiki/Gorilla_Girl_(Human_Form)>

<https://characterprofile.fandom.com/wiki/Vergil_(Devil_May_Cry)>

Según las fuentes de información, los personajes tienen las siguientes alineaciones:

- Batcow: Good
- Gorilla Girl: Good
- Vergil: Neutral evil

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>